# Expectations

In [1]:
import great_expectations as ge
from great_expectations.checkpoint import SimpleCheckpoint
import pandas as pd
import logging
import yaml
from typing import Dict, Tuple, Any

In [2]:
df = pd.read_csv("../data/01_raw/house-pricing.csv")
df_model_input = pd.read_csv("../data/05_model_input/X_train_transformed.csv")

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
gdf = ge.from_pandas(df)
num_cols = df.select_dtypes(include=['number']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

Check column amount

In [5]:
column_number = len(gdf.columns)
gdf.expect_table_column_count_to_equal(column_number)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "observed_value": 81
  },
  "success": true
}

Check if column exist

In [6]:
column_list = gdf.columns.tolist()

In [7]:
column_list_yaml = yaml.dump(column_list)
print(column_list_yaml)

- Id
- MSSubClass
- MSZoning
- LotFrontage
- LotArea
- Street
- Alley
- LotShape
- LandContour
- Utilities
- LotConfig
- LandSlope
- Neighborhood
- Condition1
- Condition2
- BldgType
- HouseStyle
- OverallQual
- OverallCond
- YearBuilt
- YearRemodAdd
- RoofStyle
- RoofMatl
- Exterior1st
- Exterior2nd
- MasVnrType
- MasVnrArea
- ExterQual
- ExterCond
- Foundation
- BsmtQual
- BsmtCond
- BsmtExposure
- BsmtFinType1
- BsmtFinSF1
- BsmtFinType2
- BsmtFinSF2
- BsmtUnfSF
- TotalBsmtSF
- Heating
- HeatingQC
- CentralAir
- Electrical
- 1stFlrSF
- 2ndFlrSF
- LowQualFinSF
- GrLivArea
- BsmtFullBath
- BsmtHalfBath
- FullBath
- HalfBath
- BedroomAbvGr
- KitchenAbvGr
- KitchenQual
- TotRmsAbvGrd
- Functional
- Fireplaces
- FireplaceQu
- GarageType
- GarageYrBlt
- GarageFinish
- GarageCars
- GarageArea
- GarageQual
- GarageCond
- PavedDrive
- WoodDeckSF
- OpenPorchSF
- EnclosedPorch
- 3SsnPorch
- ScreenPorch
- PoolArea
- PoolQC
- Fence
- MiscFeature
- MiscVal
- MoSold
- YrSold
- SaleType
- SaleCondi

In [8]:
def check_if_column_exist(gdf, column_list):
    for column in column_list:
        gdf.expect_column_to_exist(column)

In [9]:
check_if_column_exist(gdf, column_list)

Check dtypes

In [10]:
def check_dtype(gdf, columns, dtype):
    if dtype == 'numeric':
        for column in columns:
            gdf.expect_column_values_to_be_in_type_list(column, ['int64', 'float64'])
    else:
        for column in columns:
            gdf.expect_column_values_to_be_in_type_list(column, ["str"])

In [11]:
check_dtype(gdf, num_cols, dtype='numeric')
check_dtype(gdf, cat_cols, dtype='object')

Check unique values of categorical columns

In [12]:
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols_dict = {}
for column in cat_cols:
    cat_cols_dict[column] = df[column].unique().tolist()

In [13]:
cat_cols_dict

{'MSZoning': ['RL', 'RM', 'C (all)', 'FV', 'RH'],
 'Street': ['Pave', 'Grvl'],
 'Alley': [nan, 'Grvl', 'Pave'],
 'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
 'LandContour': ['Lvl', 'Bnk', 'Low', 'HLS'],
 'Utilities': ['AllPub', 'NoSeWa'],
 'LotConfig': ['Inside', 'FR2', 'Corner', 'CulDSac', 'FR3'],
 'LandSlope': ['Gtl', 'Mod', 'Sev'],
 'Neighborhood': ['CollgCr',
  'Veenker',
  'Crawfor',
  'NoRidge',
  'Mitchel',
  'Somerst',
  'NWAmes',
  'OldTown',
  'BrkSide',
  'Sawyer',
  'NridgHt',
  'NAmes',
  'SawyerW',
  'IDOTRR',
  'MeadowV',
  'Edwards',
  'Timber',
  'Gilbert',
  'StoneBr',
  'ClearCr',
  'NPkVill',
  'Blmngtn',
  'BrDale',
  'SWISU',
  'Blueste'],
 'Condition1': ['Norm',
  'Feedr',
  'PosN',
  'Artery',
  'RRAe',
  'RRNn',
  'RRAn',
  'PosA',
  'RRNe'],
 'Condition2': ['Norm',
  'Artery',
  'RRNn',
  'Feedr',
  'PosN',
  'PosA',
  'RRAn',
  'RRAe'],
 'BldgType': ['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],
 'HouseStyle': ['2Story',
  '1Story',
  '1.5Fin',
  '1.5Unf',
  

In [14]:
import yaml
result = yaml.dump(cat_cols_dict)
print(result)

Alley:
- .nan
- Grvl
- Pave
BldgType:
- 1Fam
- 2fmCon
- Duplex
- TwnhsE
- Twnhs
BsmtCond:
- TA
- Gd
- .nan
- Fa
- Po
BsmtExposure:
- 'No'
- Gd
- Mn
- Av
- .nan
BsmtFinType1:
- GLQ
- ALQ
- Unf
- Rec
- BLQ
- .nan
- LwQ
BsmtFinType2:
- Unf
- BLQ
- .nan
- ALQ
- Rec
- LwQ
- GLQ
BsmtQual:
- Gd
- TA
- Ex
- .nan
- Fa
CentralAir:
- Y
- N
Condition1:
- Norm
- Feedr
- PosN
- Artery
- RRAe
- RRNn
- RRAn
- PosA
- RRNe
Condition2:
- Norm
- Artery
- RRNn
- Feedr
- PosN
- PosA
- RRAn
- RRAe
Electrical:
- SBrkr
- FuseF
- FuseA
- FuseP
- Mix
- .nan
ExterCond:
- TA
- Gd
- Fa
- Po
- Ex
ExterQual:
- Gd
- TA
- Ex
- Fa
Exterior1st:
- VinylSd
- MetalSd
- Wd Sdng
- HdBoard
- BrkFace
- WdShing
- CemntBd
- Plywood
- AsbShng
- Stucco
- BrkComm
- AsphShn
- Stone
- ImStucc
- CBlock
Exterior2nd:
- VinylSd
- MetalSd
- Wd Shng
- HdBoard
- Plywood
- Wd Sdng
- CmentBd
- BrkFace
- Stucco
- AsbShng
- Brk Cmn
- ImStucc
- AsphShn
- Stone
- Other
- CBlock
Fence:
- .nan
- MnPrv
- GdWo
- GdPrv
- MnWw
FireplaceQu:
- .nan
- TA
-

In [15]:
def check_categorical_unique_values(gdf, dict_cat_cols):
    for column in dict_cat_cols.keys():
        gdf.expect_column_values_to_be_in_set(column, dict_cat_cols[column])

In [16]:
check_categorical_unique_values(gdf, cat_cols_dict)

Check SalePrice deviation

In [17]:
# def calculate_median_deviation(gdf, column):
#     df = gdf.copy()
#     median = df[column].median()
#     absolute_deviations = df[column].apply(lambda x: abs(x - median))
#     mad = absolute_deviations.median()
#     return mad

In [18]:
median_sales_price = df['SalePrice'].median()
median_threshold = 0.1
gdf.expect_column_median_to_be_between("SalePrice", median_sales_price*(1-median_threshold), median_sales_price*(1+median_threshold))

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "observed_value": 163000.0,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

Check columns not be null

In [19]:
def check_nulls(gdf, columns):

    for column in columns:
        gdf.expect_column_values_to_not_be_null(column)

In [20]:
check_nulls(gdf, gdf.columns)

Check if ID is unique

In [21]:
# check if id is unique
gdf.expect_column_values_to_be_unique("Id")

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [22]:
import datetime

current_year = datetime.datetime.now().year

In [23]:
gdf.expect_column_values_to_be_between("YearBuilt", 1800, current_year)

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "element_count": 1460,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [24]:
gdf.expect_column_max_to_be_between("YrSold", 1950, current_year)


{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "result": {
    "observed_value": 2010,
    "element_count": 1460,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

In [25]:
gdf[num_cols]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


In [26]:
validation = gdf.validate()

In [27]:
validation

{
  "evaluation_parameters": {},
  "meta": {
    "great_expectations_version": "0.17.1",
    "expectation_suite_name": "default",
    "run_id": {
      "run_name": null,
      "run_time": "2023-06-25T18:29:10.381107+01:00"
    },
    "batch_kwargs": {
      "ge_batch_id": "cb3fc9a9-137d-11ee-a1ba-84144d01abb6"
    },
    "batch_markers": {},
    "batch_parameters": {},
    "validation_time": "20230625T172910.381107Z",
    "expectation_suite_meta": {
      "great_expectations_version": "0.17.1"
    }
  },
  "statistics": {
    "evaluated_expectations": 291,
    "successful_expectations": 272,
    "unsuccessful_expectations": 19,
    "success_percent": 93.47079037800687
  },
  "success": false,
  "results": [
    {
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      },
      "meta": {},
      "result": {
        "observed_value": 81
      },
      "success": true,
      "expectation_config": {
        "

{'evaluation_parameters': {},
 'meta': {'great_expectations_version': '0.17.1',
  'expectation_suite_name': 'default',
  'run_id': {'run_name': None, 'run_time': '2023-06-25T18:29:10.381107+01:00'},
  'batch_kwargs': {'ge_batch_id': 'cb3fc9a9-137d-11ee-a1ba-84144d01abb6'},
  'batch_markers': {},
  'batch_parameters': {},
  'validation_time': '20230625T172910.381107Z',
  'expectation_suite_meta': {'great_expectations_version': '0.17.1'}},
 'statistics': {'evaluated_expectations': 291,
  'successful_expectations': 272,
  'unsuccessful_expectations': 19,
  'success_percent': 93.47079037800687},
 'success': False,
 'results': [{'exception_info': {'raised_exception': False,
    'exception_message': None,
    'exception_traceback': None},
   'meta': {},
   'result': {'observed_value': 81},
   'success': True,
   'expectation_config': {'meta': {},
    'expectation_type': 'expect_table_column_count_to_equal',
    'kwargs': {'value': 81, 'result_format': 'BASIC'}}},
  {'exception_info': {'raise

In [28]:
logger = logging.getLogger(__name__)

In [29]:
failed_expectations = [result for result in validation["results"] if not result["success"]]
    
logger.info(
        f"Total Expectations: {len(validation['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"Failed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )

Exception: Data Quality Validation Failed: ["Failed Expectation 1:  Expectation Type: expect_column_values_to_not_be_null  Column: LotFrontage  Details: {'element_count': 1460, 'unexpected_count': 259, 'unexpected_percent': 17.73972602739726, 'unexpected_percent_total': 17.73972602739726, 'partial_unexpected_list': []}", "Failed Expectation 2:  Expectation Type: expect_column_values_to_not_be_null  Column: Alley  Details: {'element_count': 1460, 'unexpected_count': 1369, 'unexpected_percent': 93.76712328767123, 'unexpected_percent_total': 93.76712328767123, 'partial_unexpected_list': []}", "Failed Expectation 3:  Expectation Type: expect_column_values_to_not_be_null  Column: MasVnrType  Details: {'element_count': 1460, 'unexpected_count': 8, 'unexpected_percent': 0.547945205479452, 'unexpected_percent_total': 0.547945205479452, 'partial_unexpected_list': []}", "Failed Expectation 4:  Expectation Type: expect_column_values_to_not_be_null  Column: MasVnrArea  Details: {'element_count': 1

In [49]:
import json
import os
# Create the data/08_reporting folder if it doesn't exist
folder_path = '../data/08_reporting'
os.makedirs(folder_path, exist_ok=True)

# Define the file path
file_path = os.path.join(folder_path, 'collect_errors.json')

# Save the list as a JSON file
with open(file_path, 'w') as json_file:
    json.dump(collect_errors, json_file)

print(f"The JSON file has been saved at {file_path}.")

The JSON file has been saved at ../data/08_reporting\collect_errors.json.


Node function

In [ ]:
import yaml
with open('../conf/base/parameters/data_quality.yml') as f:
    parameters = yaml.load(f, Loader=yaml.loader.SafeLoader)

logger = logging.getLogger(__name__)

def check_ranges(df: pd.DataFrame, parameters : Dict[str, Any]) -> Tuple[pd.DataFrame, Dict]:
    """

    """
    num_cols = df.select_dtypes(include=['number']).columns
    cat_cols = df.select_dtypes(include=['object']).columns
    num_columns = parameters["num_columns"]
    column_list = parameters["column_list"]
    cat_unique_values = parameters["categorical_unique_values"]
    median_sales_price = parameters["median_sales_price"]
    median_threshold = parameters["median_threshold"]

    ranges = parameters["num_quality_ranges"]
    gdf = ge.from_pandas(df)

    gdf.expect_table_column_count_to_equal(num_columns)
    check_if_column_exist(gdf, column_list)
    check_dtype(gdf, num_cols, dtype='numeric')
    check_dtype(gdf, cat_cols, dtype='object')
    check_categorical_unique_values(gdf, cat_unique_values)
    
    gdf.expect_column_median_to_be_between("SalePrice", 
                                           median_sales_price*(1-median_threshold), 
                                           median_sales_price*(1+median_threshold))
    
    check_nulls(gdf, gdf.columns)
    gdf.expect_column_values_to_be_unique("Id")

    for column in num_cols:
        gdf.expect_column_values_to_be_between(column,ranges['min'],ranges['max'])
    
    validation_results = gdf.validate()
    failed_expectations = [result for result in validation_results["results"] if not result["success"]]
    
    logger.info(
        f"Total Expectations: {len(validation_results['results'])}"
        f"Failed Expectations: {len(failed_expectations)}"
    )
    
    if failed_expectations:
        collect_errors = []
        for idx, failed_expectation in enumerate(failed_expectations, start=1):
            collect_errors.append(
                f"\nFailed Expectation {idx}:"
                f"  Expectation Type: {failed_expectation['expectation_config']['expectation_type']}"
                f"  Column: {failed_expectation['expectation_config']['kwargs']['column']}"
                f"  Details: {failed_expectation['result']}")
    
        raise Exception(
            f"Data Quality Validation Failed: {collect_errors}"
        )
   
    return df

In [37]:
parameters = yaml.load(open('../conf/base/parameters/data_quality.yml'), Loader=yaml.loader.SafeLoader)

ParserError: while parsing a block mapping
  in "../conf/base/parameters/data_quality.yml", line 1, column 1
expected <block end>, but found '<block sequence start>'
  in "../conf/base/parameters/data_quality.yml", line 93, column 3

In [15]:
check_ranges(df_model_input, parameters)

Exception: Data Quality Validation Failed: ["\nFailed Expectation 1:  Expectation Type: expect_column_values_to_be_between  Column: numerical__numerical__LotFrontage  Details: {'element_count': 1168, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 5, 'unexpected_percent': 0.4280821917808219, 'unexpected_percent_total': 0.4280821917808219, 'unexpected_percent_nonmissing': 0.4280821917808219, 'partial_unexpected_list': [12.77137639050307, 5.1397974431346505, 12.77137639050307, 5.876639548397808, 5.455586916818861]}", "\nFailed Expectation 2:  Expectation Type: expect_column_values_to_be_between  Column: numerical__numerical__LotArea  Details: {'element_count': 1168, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 18, 'unexpected_percent': 1.5410958904109588, 'unexpected_percent_total': 1.5410958904109588, 'unexpected_percent_nonmissing': 1.5410958904109588, 'partial_unexpected_list': [5.559054160841287, 8.99373898243268, 5.390553765728527, 7.41450367758798

# Terminologies

*Context*: A context in is the main object that manages the overall configuration and execution of the data expectations. It serves as a container for storing and organizing expectations, data sources, and validation results. The context allows to define, execute, and manage our data expectations.

*Validator*: A validator is responsible for evaluating expectations on a given batch of data. Validators are used to validate data against a set of predefined expectations. They help to assess data quality, perform data validation, and monitor data pipelines.

*Suite*: An Expectation Suite is a collection of expectations that define the desired properties and characteristics of our data. It serves as a set of rules against which your data can be validated. The suite contains a set of expectations that can be applied to one or more batches of data. 

*Batch*: A batch represents a subset of data that we want to evaluate against our expectations. It can be a collection of rows, a partitioned dataset, a file, a table, or any other logical grouping of data. Batches are used as inputs to validation processes and contain the data you want to validate.

*Checkpoint*: A Checkpoint is a way to operationalize data validation using Expectation Suites. It allows you to define a pipeline-like flow for performing data validation on batches of data. It helps automate the validation process by defining the steps to be executed on data batches and tracking the results.

## Installation

- Open Anaconda Prompt Terminal
- After satisfying ``!pip install great_expectations``, run ``great_expectations init`` and confirm with 'Y'
- Verify the installation by running the version ``!great_expectations --version``
- Our version used: version 0.17.1


More information: https://docs.greatexpectations.io/docs/tutorials/quickstart/

In [ ]:
context = ge.get_context(context_root_dir='../great_expectations')

In [ ]:
# Connect to data
validator = context.sources.pandas_default.read_csv("../data/01_raw/house-pricing.csv")
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
# Create expectation suite
expectation_suite_name = "house_pricing_suite"
suite = context.create_expectation_suite(expectation_suite_name=expectation_suite_name, overwrite_existing=True)

In [ ]:
# count the number of columns from validator.head()
num_columns = len(validator.head().columns)
print(f"Number of columns: {num_columns}")

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Number of columns: 81


## Expectations

Define the expectations which should be raised for validating the dataset. For the expecatations we use the information gained from the exploration step of our dataset.

In [ ]:
validator.expect_table_column_count_to_equal(81)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "observed_value": 81
  }
}

In [ ]:
validator.expect_co

## Great Expectations UI

In [ ]:
# Review and save our expectation suite
print(validator.get_expectation_suite(discard_failed_expectations=False))
validator.save_expectation_suite('../great_expectations/checkpoints/data_expectations.json')

# Create checkpoint
checkpoint = SimpleCheckpoint(name="house_pricing_checkpoint",
                              data_context=context,
                              validator=validator,
                                )

# Run checkpoint to validate data 
checkpoint_result = checkpoint.run()

# View results
validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.build_data_docs()
context.open_data_docs(resource_identifier=validation_result_identifier)

{
  "meta": {
    "great_expectations_version": "0.17.1"
  },
  "data_asset_type": null,
  "expectation_suite_name": "default",
  "ge_cloud_id": null,
  "expectations": [
    {
      "meta": {},
      "expectation_type": "expect_table_column_count_to_equal",
      "kwargs": {
        "value": 81
      }
    }
  ]
}


Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]